In [1]:
%matplotlib inline
import time
import datetime
import cv2
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

import sys
sys.path.append('../scripts')
import util

In [58]:
fname = '/home/trafferty/tmp/TTMLogStream_2017_05_17.csv'
f = open(fname, 'r')
ttm = f.readlines()
f.close()
len(ttm)

245430

In [105]:
cols = ['TS', ['measx_mm','scorex', 'measy_mm','scorey','cmeasx_mm','cscorex','cmeasy_mm','cscorey', 'x unwrap cnt', 'y unwrap cnt', 'post measx_mm', 'post measy_mm', 'post cmeasx_mm', 'post cmeasy_mm']]
col_ids = {}
for idx, col in enumerate(cols[1]):
    col_ids[col] = idx
print(col_ids)

{'cmeasx_mm': 4, 'scorey': 3, 'scorex': 1, 'post measx_mm': 10, 'post cmeasy_mm': 13, 'post measy_mm': 11, 'cscorex': 5, 'measx_mm': 0, 'cmeasy_mm': 6, 'measy_mm': 2, 'post cmeasx_mm': 12, 'x unwrap cnt': 8, 'cscorey': 7, 'y unwrap cnt': 9}


In [86]:
def parse_into_data_sets(data):
    data_ranges = []
    max_delta = 1
    start = 0
    for idx, d in enumerate(data):
        if idx == 0:
            d_prev = d
        else:
            ts1 = d_prev.split(',')[0][-23:]
            ts2 = d.split(',')[0][-23:]
            dt1 = datetime.datetime.strptime(ts1, "%Y-%m-%d %H:%M:%S.%f")
            dt2 = datetime.datetime.strptime(ts2, "%Y-%m-%d %H:%M:%S.%f")
            s1 =time.mktime(dt1.timetuple()) + (dt1.microsecond / 1000000.0)
            s2 =time.mktime(dt2.timetuple()) + (dt2.microsecond / 1000000.0)
            #delta = float(d.split(',')[0][-6:]) - float(d_prev.split(',')[0][-6:])
            delta = s2 - s1
            if delta > max_delta:
                end = idx - 1
                data_ranges.append((start, end))
                start = idx
            d_prev = d
    data_ranges.append((start, idx))
    len(data_ranges)

    data_sets = []
    for idx, data_range in enumerate(data_ranges):
        data_sets.append(data[data_range[0]:data_range[1]])
        print("Data set %d, range: %d - %d (total: %d)" % (idx, data_range[0], data_range[1], (data_range[1]-data_range[0])))
        
    for idx, ds in enumerate(data_sets):
        print("[%d] range: %s -> %s" % ( idx, ds[0].split(',')[0], ds[-1].split(',')[0]))
        
    return data_ranges, data_sets

def cleanup_and_format_data_set_old(data_sets):
    data_sets_formated = []
    for idx, d in enumerate(ds):
        data_set_new = []
        data_set = d.split(',')
        #print(idx)
        data_set_new.append(data_set[0][17:])
        data_set_new.append(float(data_set[2]))                 
        data_set_new.append(int(data_set[3]))                   
        data_set_new.append(float(data_set[4]))                 
        data_set_new.append(int(data_set[5]))                   
        data_set_new.append(float(data_set[6]))                 
        data_set_new.append(int(data_set[7]))
        data_set_new.append(float(data_set[8]))                 
        data_set_new.append(int(data_set[9]))                   
        data_set_new.append((data_set[10].count('0') - 1) * -1)
        data_set_new.append((data_set[11].count('0') - 1) * -1)
        data_set_new.append(float(data_set[13]))                
        data_set_new.append(float(data_set[14]))                
        data_set_new.append(float(data_set[15]))                
        data_set_new.append(float(data_set[16]))                
        data_set_new.append(float(data_set[18]))                
        data_set_new.append(int(data_set[19]))                  
        data_set_new.append(float(data_set[20]))                
        data_set_new.append(int(data_set[21]))                  
        data_set_new.append(float(data_set[22]))                
        data_set_new.append(int(data_set[23]))                  
        data_set_new.append(float(data_set[24]))                
        data_set_new.append(int(data_set[25]))                  
        data_set_new.append((data_set[26].count('0') - 1) * -1)
        data_set_new.append((data_set[27].count('0') - 1) * -1)
        data_set_new.append(float(data_set[29]))                
        data_set_new.append(float(data_set[30]))                
        data_set_new.append(float(data_set[31]))                
        data_set_new.append(float(data_set[32]))                
        data_set_new.append(float(data_set[34]))                
        data_set_new.append(int(data_set[35]))                  
        data_set_new.append(float(data_set[36]))                
        data_set_new.append(int(data_set[37]))                  
        data_set_new.append(float(data_set[38]))                
        data_set_new.append(int(data_set[39]))                  
        data_set_new.append(float(data_set[40]))                
        data_set_new.append(int(data_set[41]))                  
        data_set_new.append((data_set[42].count('0') - 1) * -1)
        data_set_new.append((data_set[43].count('0') - 1) * -1)
        data_set_new.append(float(data_set[45]))                
        data_set_new.append(float(data_set[46]))                
        data_set_new.append(float(data_set[47]))                
        data_set_new.append(float(data_set[48]))                
        data_set_new.append(float(data_set[50]))                
        data_set_new.append(int(data_set[51]))                  
        data_set_new.append(float(data_set[52]))                
        data_set_new.append(int(data_set[53]))                  
        data_set_new.append(float(data_set[54]))                
        data_set_new.append(int(data_set[55]))                  
        data_set_new.append(float(data_set[56]))                
        data_set_new.append(int(data_set[57]))                  
        data_set_new.append((data_set[58].count('0') - 1) * -1)
        data_set_new.append((data_set[59].count('0') - 1) * -1)
        data_set_new.append(float(data_set[61]))                
        data_set_new.append(float(data_set[62]))                
        data_set_new.append(float(data_set[63]))                
        data_set_new.append(float(data_set[64]))
        data_sets_formated.append(data_set_new)
    return data_sets_formated
        
def cleanup_and_format_data_set(data_sets):
    data_sets_formated = []
    for idx, data_set_str in enumerate(data_sets):
        data_set_new = []
        data_set = data_set_str.split(',')
        #print(idx)
        
        data_set_new.append(data_set[0][17:])

        cam1_data = []
        cam1_data.append(float(data_set[2]))                 
        cam1_data.append(int(data_set[3]))                   
        cam1_data.append(float(data_set[4]))                 
        cam1_data.append(int(data_set[5]))                   
        cam1_data.append(float(data_set[6]))                 
        cam1_data.append(int(data_set[7]))
        cam1_data.append(float(data_set[8]))                 
        cam1_data.append(int(data_set[9]))                   
        cam1_data.append((data_set[10].count('0') - 1) * -1)
        cam1_data.append((data_set[11].count('0') - 1) * -1)
        cam1_data.append(float(data_set[13]))                
        cam1_data.append(float(data_set[14]))                
        cam1_data.append(float(data_set[15]))                
        cam1_data.append(float(data_set[16]))
        data_set_new.append(cam1_data)
        
        cam2_data = []
        cam2_data.append(float(data_set[18]))                
        cam2_data.append(int(data_set[19]))   
        cam2_data.append(float(data_set[20]))                
        cam2_data.append(int(data_set[21]))                  
        cam2_data.append(float(data_set[22]))                
        cam2_data.append(int(data_set[23]))                  
        cam2_data.append(float(data_set[24]))                
        cam2_data.append(int(data_set[25]))                  
        cam2_data.append((data_set[26].count('0') - 1) * -1)
        cam2_data.append((data_set[27].count('0') - 1) * -1)
        cam2_data.append(float(data_set[29]))                
        cam2_data.append(float(data_set[30]))                
        cam2_data.append(float(data_set[31]))                
        cam2_data.append(float(data_set[32]))
        data_set_new.append(cam2_data)
        
        cam3_data = []
        cam3_data.append(float(data_set[34]))                
        cam3_data.append(int(data_set[35]))                  
        cam3_data.append(float(data_set[36]))                
        cam3_data.append(int(data_set[37]))                  
        cam3_data.append(float(data_set[38]))                
        cam3_data.append(int(data_set[39]))                  
        cam3_data.append(float(data_set[40]))                
        cam3_data.append(int(data_set[41]))                  
        cam3_data.append((data_set[42].count('0') - 1) * -1)
        cam3_data.append((data_set[43].count('0') - 1) * -1)
        cam3_data.append(float(data_set[45]))                
        cam3_data.append(float(data_set[46]))                
        cam3_data.append(float(data_set[47]))                
        cam3_data.append(float(data_set[48]))
        data_set_new.append(cam3_data)
        
        cam4_data = []
        cam4_data.append(float(data_set[50]))                
        cam4_data.append(int(data_set[51]))                  
        cam4_data.append(float(data_set[52]))                
        cam4_data.append(int(data_set[53]))                  
        cam4_data.append(float(data_set[54]))                
        cam4_data.append(int(data_set[55]))                  
        cam4_data.append(float(data_set[56]))                
        cam4_data.append(int(data_set[57]))                  
        cam4_data.append((data_set[58].count('0') - 1) * -1)
        cam4_data.append((data_set[59].count('0') - 1) * -1)
        cam4_data.append(float(data_set[61]))                
        cam4_data.append(float(data_set[62]))                
        cam4_data.append(float(data_set[63]))                
        cam4_data.append(float(data_set[64]))
        data_set_new.append(cam4_data)
        
        data_sets_formated.append(data_set_new)
    return data_sets_formated

In [87]:
data_ranges, data_sets_rough = parse_into_data_sets(ttm)

Data set 0, range: 0 - 11708 (total: 11708)
Data set 1, range: 11709 - 23305 (total: 11596)
Data set 2, range: 23306 - 35013 (total: 11707)
Data set 3, range: 35014 - 46612 (total: 11598)
Data set 4, range: 46613 - 58322 (total: 11709)
Data set 5, range: 58323 - 69917 (total: 11594)
Data set 6, range: 69918 - 81626 (total: 11708)
Data set 7, range: 81627 - 92889 (total: 11262)
Data set 8, range: 92890 - 104602 (total: 11712)
Data set 9, range: 104603 - 116312 (total: 11709)
Data set 10, range: 116313 - 127916 (total: 11603)
Data set 11, range: 127917 - 139627 (total: 11710)
Data set 12, range: 139628 - 151222 (total: 11594)
Data set 13, range: 151223 - 162924 (total: 11701)
Data set 14, range: 162925 - 174627 (total: 11702)
Data set 15, range: 174628 - 175947 (total: 1319)
Data set 16, range: 175948 - 187546 (total: 11598)
Data set 17, range: 187547 - 198925 (total: 11378)
Data set 18, range: 198926 - 210526 (total: 11600)
Data set 19, range: 210527 - 222238 (total: 11711)
Data set 20,

In [83]:
c = {}
for idx, ds in enumerate(data_sets_rough):
    for d in ds:
        cnt = d.count(',')
        if cnt not in c:
            c[cnt] = 1
        else:
            c[cnt] += 1
print(c)

{64: 245408}


In [88]:
data_sets = []
for idx, ds in enumerate(data_sets_rough):
    data_sets.append(cleanup_and_format_data_set(ds))

In [82]:
c = {}
for idx, ds in enumerate(data_sets):
    for d in ds:
        #cnt = d.count(',')
        cnt = len(d)
        if cnt not in c:
            c[cnt] = 1
        else:
            c[cnt] += 1
print(c)

{57: 245408}


In [114]:
numCams = 4

In [113]:
ds = data_sets[5]  #Xiaoming thinks this one had spike...

for col_id in col_ids.keys():
    print("%s [%d]: %s" % (col_id, col_ids[col_id], str(ds[15][4][col_ids[col_id]])))
    #print("%s: " % (col_id))
ds[15]

cmeasx_mm [4]: -0.002479
scorey [3]: 0
scorex [1]: 0
post measx_mm [10]: -0.0023709
post cmeasy_mm [13]: -0.000621
post measy_mm [11]: -0.0005212
cscorex [5]: 0
measx_mm [0]: -0.0003709
cmeasy_mm [6]: -0.000621
measy_mm [2]: -0.0005212
post cmeasx_mm [12]: -0.002479
x unwrap cnt [8]: 1
cscorey [7]: 0
y unwrap cnt [9]: 0


['2017-05-17 09:15:49.815',
 [0.001, 1, 0.001, 1, 0.0, 1, 0.0, 1, -1, -1, nan, nan, 0.0, 0.0],
 [0.001, 1, 0.001, 1, 0.0, 1, 0.0, 1, -1, -1, nan, nan, 0.0, 0.0],
 [0.001, 1, 0.001, 1, 0.0, 1, 0.0, 1, -1, -1, nan, nan, 0.0, 0.0],
 [-0.0003709,
  0,
  -0.0005212,
  0,
  -0.002479,
  0,
  -0.000621,
  0,
  1,
  0,
  -0.0023709,
  -0.0005212,
  -0.002479,
  -0.000621]]

In [121]:
TTM_MoirePeriod_mm = 0.001
TTM_CoarseOffset_mm = 0

def unwrap(meas_mm, cmeas_mm, cscore):
    cnt = 0
    if cscore > 0:
        meas_mm = -np.nan
    else:
        err = cmeas_mm + TTM_CoarseOffset_mm - meas_mm
        while (np.fabs(err) > 0.5*TTM_MoirePeriod_mm) and (cnt < 4):
            if err > 0.0:
                meas_mm += TTM_MoirePeriod_mm
            else:
                meas_mm -= TTM_MoirePeriod_mm

            err = cmeas_mm + TTM_CoarseOffset_mm - meas_mm
            cnt += 1
    return cnt, meas_mm

In [122]:
unwrap(-0.0003709, -0.002479, 0)

(2, -0.0023709)

In [123]:
unwrap(ds[15][4][col_ids['measx_mm']], ds[15][4][col_ids['cmeasx_mm']], ds[15][4][col_ids['cscorex']])


(2, -0.0023709)

In [ ]:
plt.figure(figsize=(10*2,5))
plt.plot(chan1[190:], color='b')
plt.plot(chan2[190:], color='g')
plt.plot(chan3[190:], color='r')
# plt.vlines(lt_peakIdx, 0, np.max(patch_diff) + 25, color='r')
# plt.vlines(rt_peakIdx, 0, np.max(patch_diff) + 25, color='r')
plt.title('HLS mean values from center ROI')
